# Connecting to DB in RDS Guide
### Miles Child - November 2022
#### Northeastern Trading Systems

#### Starting DB and importing necessary packages

Once you have created your database in RDS, make sure that it is configured to be publicly accessible and that you've attached a security group that allows inbound/outbound access from 'All Traffic' on both IPv4 and IPv6. Save the login credentials of the DB as soon as you launch the instance.

Next, navigate to your local python environment and install the pymysql package. This process can be very messy if done incorrectly the first time, so make sure you do the following:

For Python 2.x:
- sudo pip install pymysql.

For Python 3.x:
- sudo pip3 install pymysql.

For either running on Anaconda:
- sudo conda install pymysql.

If that didn't work try APT:
- sudo apt-get install pymysql.

credit: https://stackoverflow.com/a/49354387/20355075

Finally, 'import pymysql' in your project before continuing.

In [1]:
# imports
import pymysql
from config import *  # credentials stored here

Now, establish a connection to the db by using pymysql.connect(host=<'someurl.rds.amazonaws.com'>, user=<'username'>, password=<'password'>).

Create a cursor, which can essentially be regarded as an object that executes your queries (written locally) on the DB, stores the result in-memory, and returns it back to this project. Check that the connection and cursor have been made properly by checking the version of the db.

In [2]:
connection = pymysql.connect(host=RDS_HOSTNAME, user=RDS_USER, password=RDS_PASSWORD, database='nts_lightning_db_v2', autocommit=True)
cursor = connection.cursor()
cursor.execute('select version()')

1

In [3]:
#cursor.execute('use nts_lightning_db_v2')
cursor.execute('show tables')

4

In [8]:
cursor.execute('select * from strategy')

0

After successfully establishing a connection to the RDS instance, create a DB and add the neccessary tables/entities/attributes.

In [ ]:
# Create a new database
cursor.execute("CREATE DATABASE nts_lightning_db;")
cursor.execute(f"CREATE USER 'webapp'@'%' IDENTIFIED BY {DOCKER_DB_PASSWORD};")
cursor.execute("GRANT ALL PRIVILEGES ON nts_lightning_db.* TO 'webapp'@'%';")
cursor.execute("FLUSH PRIVILEGES;")
cursor.execute("USE nts_lightning_db;")

In [ ]:
# Create the tables
cursor.execute("""CREATE TABLE strategy (
    strategy_name varchar(50),
    strategy_id integer AUTO_INCREMENT NOT NULL,
    documentation_link varchar(50),
    launch_date datetime,
    termination_date datetime,
    PRIMARY KEY (strategy_id)
);""")

In [ ]:
cursor.execute("""CREATE TABLE trade (
    trade_id integer UNIQUE NOT NULL,
    strategy_id integer,
    open_time datetime,
    close_time datetime,
    PRIMARY KEY (trade_id),
    CONSTRAINT fk_1 
        FOREIGN KEY (strategy_id) REFERENCES strategy(strategy_id)
        ON UPDATE cascade ON DELETE restrict
);""")

In [ ]:
cursor.execute("""CREATE TABLE trade_leg (
    leg_no integer NOT NULL,
    trade_id integer NOT NULL,
    contract varchar(50),
    open_time datetime,
    close_time datetime,
    PRIMARY KEY(leg_no, trade_id),
    CONSTRAINT fk_2
        FOREIGN KEY (trade_id) REFERENCES trade (trade_id)
        ON UPDATE restrict ON DELETE restrict
);""")

In [ ]:
cursor.execute("""CREATE TABLE fill (
    fill_id integer NOT NULL,
    leg_no integer NOT NULL,
    trade_id integer NOT NULL,
    contract varchar(50),
    qty integer,
    avg float(32),
    placement_time datetime,
    filled_time datetime,
    PRIMARY KEY (fill_id),
    CONSTRAINT fk_3
        FOREIGN KEY (leg_no, trade_id) REFERENCES trade_leg (leg_no, trade_id)
        ON UPDATE cascade ON DELETE restrict
);""")

After creating the database architecture, insert testing (or historical) data into the tables.

note: mockaroo is good for generating dummy data https://www.mockaroo.com/

In [ ]:
# Strategy Dummy Data
cursor.execute("""INSERT INTO strategy (strategy_name,strategy_id,documentation_link,launch_date,termination_date) 
values 
('MeanReversionLunchBreak', '1', 'https://github.com/QuantConnect/Lean/blob/master/Algorithm.Python/Alphas/MeanReversionLunchBreakAlpha.py',
 '2022-11-20 00:00:00', NULL);""")


In [ ]:
# Trade Dummy Data
cursor.execute("""insert into trade (trade_id, strategy_id, open_time, close_time) 
values 
('000000001', '1', '2022-11-21 12:00:00', '2022-11-21 12:25:00');
""")
cursor.execute("""insert into trade (trade_id, strategy_id, open_time, close_time) 
values 
('000000002', '1', '2022-11-22 11:49:00', '2022-11-22 12:03:00');
""")

In [ ]:
# Trade Leg Dummy Data
cursor.execute("""insert into trade_leg (leg_no, trade_id, contract, open_time, close_time) 
values 
('1', '000000001', 'SPY', '2022-11-21 12:00:00', '2022-11-21 12:25:00');
""")

# note: the below data represents the following trade:
# Seller of AAPL 148/150 Dec. 02 call spread
cursor.execute("""insert into trade_leg (leg_no, trade_id, contract, open_time, close_time) 
values 
('1', '000000002', 'AAPL221202C00148000', '2022-11-22 11:49:00', '2022-11-22 12:03:00');
""")
cursor.execute("""insert into trade_leg (leg_no, trade_id, contract, open_time, close_time) 
values 
('2', '000000002', 'AAPL221202C00150000', '2022-11-22 11:49:00', '2022-11-22 12:03:00');
""")

In [ ]:
# Fill Dummy Data
cursor.execute("""insert into fill (fill_id, leg_no, trade_id, contract, qty, avg, placement_time, filled_time) 
values 
('000000001', '1', '000000001', 'SPY', '-100', '400.00', '2022-11-21 12:00:00', '2022-11-21 12:00:01');
""")

cursor.execute("""insert into fill (fill_id, leg_no, trade_id, contract, qty, avg, placement_time, filled_time) 
values 
('000000002', '1', '000000001', 'SPY', '100', '397.00', '2022-11-21 12:24:58', '2022-11-21 12:25:00');
""")


# note: the below data represents the following trade:
# Seller of AAPL 148/150 Dec. 02 call spread 2.50 @ 2.00, 10 x 10

cursor.execute("""insert into fill (fill_id, leg_no, trade_id, contract, qty, avg, placement_time, filled_time) 
values 
('000000003', '1', '000000002', 'AAPL221202C00148000', '-10', '2.50', '2022-11-22 11:49:00', '2022-11-22 11:49:01');
""")

cursor.execute("""insert into fill (fill_id, leg_no, trade_id, contract, qty, avg, placement_time, filled_time) 
values 
('000000004', '2', '000000002', 'AAPL221202C00150000', '10', '2.00', '2022-11-21 11:49:00', '2022-11-21 11:49:01');
""")

cursor.execute("""insert into fill (fill_id, leg_no, trade_id, contract, qty, avg, placement_time, filled_time) 
values 
('000000005', '1', '000000002', 'AAPL221202C00148000', '10', '2.25', '2022-11-22 12:02:59', '2022-11-22 12:02:59');
""")

cursor.execute("""insert into fill (fill_id, leg_no, trade_id, contract, qty, avg, placement_time, filled_time) 
values 
('000000006', '2', '000000002', 'AAPL221202C00150000', '-10', '1.90', '2022-11-22 12:02:59', '2022-11-22 12:03:00');
""")

In [3]:
# Testing that all dummy data was inserted properly and exists in the db.

cursor.execute('SELECT * from fill;')
col_headers = [x[0] for x in cursor.description]
json_data = []
the_data = cursor.fetchall()
for row in the_data:
    json_data.append(dict(zip(col_headers, row)))
json_data

[]

In [4]:
# Testing get_strategy_info

import pandas as pd

cursor.execute('SELECT * from strategy;')
col_headers = [x[0] for x in cursor.description]
json_data = []
the_data = cursor.fetchall()
for row in the_data:
    json_data.append(dict(zip(col_headers, row)))
json_data

[]

In [ ]:
# Testing get_trade_info

# TODO: FIX QUERY

cursor.execute("""
SELECT * from trade join trade_leg on trade.trade_id = trade_leg.trade_id join fill on trade_leg.leg_no = fill.leg_no and trade_leg.trade_id = fill.trade_id
where trade.trade_id = '000000002'
order by trade_leg.leg_no, fill.placement_time;
""")
col_headers = [x[0] for x in cursor.description]
json_data = []
the_data = cursor.fetchall()
for row in the_data:
    json_data.append(dict(zip(col_headers, row)))
df = pd.DataFrame(json_data)
df


In [ ]:
# Testing get_trade_leg_info
cursor.execute("""
Select * from trade_leg join fill on trade_leg.leg_no = fill.leg_no and trade_leg.trade_id = fill.trade_id
where trade_leg.trade_id = '000000002' and trade_leg.leg_no = '1'
order by fill.placement_time;
""")
col_headers = [x[0] for x in cursor.description]
json_data = []
the_data = cursor.fetchall()
for row in the_data:
    json_data.append(dict(zip(col_headers, row)))
df = pd.DataFrame(json_data)
df

In [ ]:
# Testing get_fill_info
cursor.execute("""
Select * from fill where fill.fill_id = '000000002';
""")
col_headers = [x[0] for x in cursor.description]
json_data = []
the_data = cursor.fetchall()
for row in the_data:
    json_data.append(dict(zip(col_headers, row)))
json_data
#df = pd.DataFrame(json_data)
#df

In [ ]:
# Testing get_open_trades
strategy = 'MeanReversionLunchBreak'
cursor.execute(f"""
Select * from strategy 
join trade on strategy.strategy_id = trade.strategy_id 
join trade_leg on trade.trade_id = trade_leg.trade_id 
join fill on trade_leg.leg_no = fill.leg_no and trade_leg.trade_id = fill.trade_id
where strategy.strategy_name = '{strategy}' and trade_leg.close_time is null
order by fill.placement_time;
""")
col_headers = [x[0] for x in cursor.description]
json_data = []
the_data = cursor.fetchall()
for row in the_data:
    json_data.append(dict(zip(col_headers, row)))
json_data

In [ ]:
# Testing get_historical_trades

In [21]:
text = """INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (927820906,1,998325118,'ATHM',237,114.37,'2022-02-28 17:07:40','2022-02-28 17:07:48');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (443930392,1,300585899,'COF^D',226,47.96,'2022-02-14 21:17:37','2022-02-14 21:17:45');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (885119335,1,608530551,'GAB',270,455.2,'2022-08-18 11:32:01','2022-08-18 11:32:11');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (244092481,1,375500853,'CRS',144,246.85,'2022-10-22 09:17:19','2022-10-22 09:17:19');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (980969611,1,274391853,'EQGP',337,355.33,'2022-10-19 09:02:55','2022-10-19 09:03:01');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (526741529,1,820638991,'VOD',491,296.26,'2021-12-16 01:18:17','2021-12-16 01:18:27');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (182235790,1,872630201,'RDC',217,349.66,'2022-10-17 23:03:28','2022-10-17 23:03:34');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (707455414,1,956707872,'PFGC',110,108.39,'2022-01-26 07:14:45','2022-01-26 07:14:48');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (313631909,1,923792033,'RLGY',289,475.24,'2022-09-18 11:01:49','2022-09-18 11:01:54');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (155518072,1,250415049,'RVLT',240,236.65,'2022-09-16 15:05:30','2022-09-16 15:05:34');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (403896689,1,459178493,'BOSC',473,341.85,'2022-04-29 03:59:44','2022-04-29 03:59:53');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (834441178,1,329626266,'NUROW',277,322.99,'2022-04-16 07:30:15','2022-04-16 07:30:17');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (807505932,1,466560851,'NUTR',414,265.64,'2022-09-16 15:34:01','2022-09-16 15:34:01');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (112557235,1,965514288,'PHO',150,300.13,'2021-12-17 10:43:52','2021-12-17 10:43:52');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (752136878,1,912632634,'HVT.A',310,33.63,'2022-07-11 15:43:05','2022-07-11 15:43:05');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (897305709,1,750566235,'FH',275,166.68,'2022-01-04 10:30:14','2022-01-04 10:30:22');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (052740558,1,491606976,'TSRI',444,180.27,'2022-08-19 20:00:32','2022-08-19 20:00:34');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (477900010,1,116185127,'JPM^G',118,360.67,'2022-10-05 10:54:49','2022-10-05 10:54:57');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (725806582,1,427437569,'RMT',121,62.57,'2022-11-19 19:26:14','2022-11-19 19:26:18');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (876568229,1,162906379,'NTZ',134,234.37,'2022-07-16 21:50:07','2022-07-16 21:50:14');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (466765543,1,323945588,'LHO^J',416,96.83,'2022-02-18 03:47:46','2022-02-18 03:47:49');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (077706377,1,004555319,'EMCF',425,387.68,'2022-06-01 00:12:28','2022-06-01 00:12:31');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (369519600,1,739016238,'PBCTP',355,70.92,'2022-03-01 19:46:48','2022-03-01 19:46:54');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (368358573,1,334604287,'GLMD',478,41.13,'2022-05-25 14:55:37','2022-05-25 14:55:42');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (528098932,1,614004145,'SPE^B',293,316.07,'2022-01-07 22:54:48','2022-01-07 22:54:54');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (917103057,1,114541558,'PIO',495,422.78,'2022-07-13 22:49:31','2022-07-13 22:49:34');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (020572043,1,603296925,'UNM',401,134.26,'2022-02-02 23:43:14','2022-02-02 23:43:21');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (492246489,1,060637430,'CSL',139,296.37,'2022-03-10 10:58:36','2022-03-10 10:58:40');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (822071208,1,319716523,'SUNS',408,293.16,'2022-11-14 01:49:59','2022-11-14 01:50:09');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (120119543,1,105425552,'ETX',483,378.97,'2022-11-07 13:03:30','2022-11-07 13:03:40');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (760392402,1,412606721,'MFSF',372,470.01,'2022-06-19 00:58:29','2022-06-19 00:58:34');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (690751344,1,668865077,'HTHT',227,107.92,'2022-05-15 20:19:04','2022-05-15 20:19:11');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (037797072,1,667385692,'MTGEP',480,226.39,'2022-03-01 06:20:40','2022-03-01 06:20:47');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (614953177,1,809629504,'AUPH',349,43.18,'2022-01-08 22:49:06','2022-01-08 22:49:16');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (169227022,1,826060268,'SPAN',482,286.78,'2022-11-27 03:49:15','2022-11-27 03:49:15');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (211344511,1,221275029,'TRIL',341,116.04,'2022-06-26 06:12:23','2022-06-26 06:12:31');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (523643983,1,030490782,'GEF.B',307,396.08,'2022-04-07 10:12:04','2022-04-07 10:12:07');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (217561646,1,021904830,'ANCX',251,136.23,'2022-01-03 03:10:20','2022-01-03 03:10:21');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (055779593,1,429373670,'ISBC',163,307.8,'2022-01-30 11:32:37','2022-01-30 11:32:44');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (185606256,1,249797132,'WLB',168,384.63,'2022-02-03 13:55:35','2022-02-03 13:55:44');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (797825042,1,769321991,'JEC',448,203.4,'2022-08-02 03:38:36','2022-08-02 03:38:37');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (590210089,1,499684733,'BGE^B',383,473.7,'2022-09-09 18:54:18','2022-09-09 18:54:22');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (797869463,1,082927608,'LALT',259,437.28,'2021-12-18 01:34:57','2021-12-18 01:35:07');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (091656733,1,512536957,'ADRU',227,4.74,'2022-07-09 23:30:26','2022-07-09 23:30:31');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (893495947,1,493467432,'LJPC',432,495.59,'2022-05-19 09:46:15','2022-05-19 09:46:24');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (167376516,1,142660041,'NI',394,162.43,'2022-11-14 17:45:54','2022-11-14 17:46:00');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (504882769,1,628189671,'UBFO',135,320.52,'2022-08-14 16:59:55','2022-08-14 16:59:59');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (440990429,1,331633006,'ADHD',301,81.42,'2022-08-24 03:36:40','2022-08-24 03:36:44');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (972896042,1,516575097,'MRK',174,106.33,'2022-02-24 21:22:41','2022-02-24 21:22:42');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (167810540,1,318906343,'ADBE',394,349.25,'2022-04-03 08:58:54','2022-04-03 08:58:57');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (245928122,1,985885601,'WBKC',452,231.98,'2022-03-18 22:11:16','2022-03-18 22:11:17');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (016642096,1,918081156,'L',291,311.5,'2022-03-07 04:56:39','2022-03-07 04:56:40');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (417819399,1,778764675,'WABC',273,417.83,'2022-01-05 00:55:11','2022-01-05 00:55:18');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (453526534,1,026355223,'JE',423,429.67,'2021-12-31 09:21:42','2021-12-31 09:21:52');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (195982840,1,592084726,'BXE',397,487.05,'2021-12-21 22:54:19','2021-12-21 22:54:27');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (631874129,1,942431792,'TWNKW',354,377.85,'2022-05-23 03:38:30','2022-05-23 03:38:39');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (804576161,1,460130244,'LGF.B',265,244.18,'2022-01-18 08:02:09','2022-01-18 08:02:15');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (233299323,1,262348555,'BICK',280,310.87,'2022-08-21 00:50:51','2022-08-21 00:50:54');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (548328404,1,282079415,'NEE^C',392,371.15,'2022-05-01 16:51:10','2022-05-01 16:51:20');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (020765247,1,324101738,'QQQX',423,99.74,'2022-10-14 18:52:55','2022-10-14 18:52:57');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (671498856,1,676609785,'SMIT',131,223.04,'2022-06-13 03:35:28','2022-06-13 03:35:33');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (494976025,1,580798972,'LBIO',328,268.28,'2022-05-09 18:14:40','2022-05-09 18:14:42');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (889631319,1,311914677,'CACC',111,309.3,'2022-09-21 16:20:15','2022-09-21 16:20:24');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (091599283,1,657301532,'PACB',468,110.44,'2022-04-24 02:18:14','2022-04-24 02:18:16');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (690392269,1,275156034,'BSPM',272,50.65,'2022-06-06 14:32:18','2022-06-06 14:32:22');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (092057469,1,909594814,'MEOH',285,485.87,'2022-05-13 14:28:01','2022-05-13 14:28:08');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (578897576,1,109908460,'SIMO',252,331.68,'2022-04-13 20:02:53','2022-04-13 20:03:01');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (184339359,1,532339969,'PTR',109,170.55,'2021-12-16 03:05:16','2021-12-16 03:05:21');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (761763075,1,228065620,'HWBK',246,278.94,'2022-02-11 04:58:42','2022-02-11 04:58:49');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (522427512,1,282075074,'ADXS',469,189.89,'2022-09-26 13:25:02','2022-09-26 13:25:05');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (073902352,1,998104368,'XRX',235,33.27,'2022-05-25 23:59:27','2022-05-25 23:59:32');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (271124762,1,662214680,'GBLIL',119,198.09,'2022-01-12 04:38:04','2022-01-12 04:38:13');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (532343372,1,076255005,'KNL',340,398.28,'2022-05-26 08:20:50','2022-05-26 08:20:51');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (970610132,1,834471688,'YUME',357,248.64,'2022-09-13 04:20:02','2022-09-13 04:20:02');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (478817759,1,239847666,'PTIE',498,22.51,'2022-01-26 19:00:11','2022-01-26 19:00:18');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (671923419,1,818775600,'LRAD',134,296.71,'2022-01-18 21:40:54','2022-01-18 21:40:57');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (070269322,1,550413487,'WAFDW',424,128.72,'2021-12-08 01:27:48','2021-12-08 01:27:48');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (838429149,1,713394497,'AVT',216,54.99,'2022-10-20 08:27:51','2022-10-20 08:27:57');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (888206460,1,726957289,'GAIA',220,368.26,'2022-08-22 05:46:18','2022-08-22 05:46:27');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (947223919,1,574906844,'OPP',310,199.44,'2021-12-08 13:47:54','2021-12-08 13:47:58');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (793413177,1,311161537,'COLL',107,92.66,'2022-06-09 18:23:01','2022-06-09 18:23:10');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (712980594,1,325819132,'WYN',465,220.02,'2022-01-18 13:00:01','2022-01-18 13:00:06');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (536040851,1,746446204,'SNP',232,253.29,'2022-05-08 11:08:53','2022-05-08 11:08:53');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (559663039,1,977263673,'KRC^H',163,434.74,'2022-09-10 19:24:04','2022-09-10 19:24:04');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (422455378,1,743118607,'MNST',450,438.53,'2022-02-15 05:32:38','2022-02-15 05:32:40');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (081525401,1,675416933,'WLH',199,321.4,'2022-03-18 18:22:09','2022-03-18 18:22:12');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (030275677,1,397180433,'ALX',346,398.16,'2021-12-19 16:12:07','2021-12-19 16:12:11');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (228617351,1,929555760,'LBIX',183,304.51,'2022-11-06 09:29:04','2022-11-06 09:29:09');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (393858172,1,139480550,'BSAC',166,238.24,'2022-04-21 09:48:50','2022-04-21 09:49:00');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (273925129,1,171327069,'TPVZ',288,342.04,'2022-01-17 18:45:26','2022-01-17 18:45:28');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (551894814,1,952756584,'CHMI',372,463.07,'2022-10-02 13:16:11','2022-10-02 13:16:20');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (255524125,1,044409115,'WAGE',407,413.63,'2022-11-12 16:47:16','2022-11-12 16:47:25');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (118032721,1,835059712,'NGHCN',274,150.21,'2022-06-11 18:16:58','2022-06-11 18:17:04');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (086785546,1,693880970,'IPWR',318,316.32,'2022-05-22 01:09:41','2022-05-22 01:09:46');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (135885569,1,660458440,'PBR.A',375,163.54,'2022-01-29 14:20:37','2022-01-29 14:20:39');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (996156507,1,381145924,'NEWM',291,381.06,'2022-06-21 07:39:48','2022-06-21 07:39:50');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (760086658,1,476704648,'OCUL',193,134.72,'2022-02-16 04:35:59','2022-02-16 04:36:09');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (053892004,1,004965884,'LORL',203,84.99,'2022-06-18 06:27:55','2022-06-18 06:28:05');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (953854241,1,739502250,'MON',358,93.11,'2022-07-14 06:01:56','2022-07-14 06:02:01');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (587714322,1,796284004,'CWBC',210,196.57,'2022-11-22 22:40:11','2022-11-22 22:40:19');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (418384313,1,231075256,'KWR',426,308.48,'2022-04-11 23:42:21','2022-04-11 23:42:24');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (956235792,1,642506826,'ECR',357,226.81,'2022-09-16 18:15:55','2022-09-16 18:16:01');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (102801580,1,753593819,'CELH',113,64.93,'2022-07-23 11:55:06','2022-07-23 11:55:14');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (780870976,1,921476346,'WYNN',365,184.12,'2022-05-17 10:37:56','2022-05-17 10:37:57');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (891473126,1,017682620,'SQM',215,195.79,'2022-09-12 04:02:31','2022-09-12 04:02:38');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (515415161,1,561877915,'CTB',229,173.65,'2022-08-25 08:45:08','2022-08-25 08:45:14');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (352455293,1,761782348,'FNFV',311,218.72,'2022-11-30 04:31:41','2022-11-30 04:31:45');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (631531647,1,034921263,'PDLI',147,68.62,'2022-10-03 06:17:07','2022-10-03 06:17:14');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (519045641,1,798894307,'EMES',297,198.66,'2022-03-29 22:22:41','2022-03-29 22:22:50');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (284655130,1,708430465,'WAB',378,33.7,'2022-09-28 05:35:15','2022-09-28 05:35:15');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (482851292,1,647852592,'HRS',315,106.75,'2022-06-19 04:26:41','2022-06-19 04:26:45');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (860993691,1,907014647,'KREF',419,377.79,'2022-04-17 05:07:12','2022-04-17 05:07:18');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (210114829,1,747615399,'JP',444,279.4,'2022-02-11 21:32:16','2022-02-11 21:32:22');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (629167623,1,715809122,'RRC',112,23.37,'2022-09-10 06:32:24','2022-09-10 06:32:29');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (475624648,1,167207503,'BLH',177,89.95,'2022-08-14 04:33:52','2022-08-14 04:33:58');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (863695135,1,646937032,'WCST',415,160.56,'2022-10-17 17:00:31','2022-10-17 17:00:35');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (724906179,1,539056113,'B',212,102.47,'2022-04-14 18:20:14','2022-04-14 18:20:15');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (186393315,1,731317912,'ARW',175,330.79,'2022-01-19 06:09:54','2022-01-19 06:09:58');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (803731002,1,074780072,'AWP',123,349.82,'2022-02-16 23:29:42','2022-02-16 23:29:51');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (710786732,1,130854604,'NCR',479,269.0,'2022-04-12 20:49:54','2022-04-12 20:49:57');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (684624544,1,412202357,'GHC',243,57.46,'2022-08-31 05:54:39','2022-08-31 05:54:48');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (830036125,1,754086541,'VLRS',371,178.17,'2022-07-09 12:47:12','2022-07-09 12:47:17');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (862917833,1,385740146,'CYD',484,314.47,'2022-07-16 14:44:01','2022-07-16 14:44:03');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (165755639,1,206054292,'FTAI',168,96.63,'2022-07-09 16:50:28','2022-07-09 16:50:34');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (100804200,1,288431752,'ARAY',103,119.17,'2022-08-05 01:59:28','2022-08-05 01:59:32');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (563997730,1,557782593,'ORI',356,392.61,'2022-11-19 18:29:23','2022-11-19 18:29:24');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (949719739,1,119429397,'CPN',100,197.91,'2021-12-22 12:28:03','2021-12-22 12:28:10');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (643830819,1,848733680,'CYCC',457,345.12,'2022-02-01 02:00:27','2022-02-01 02:00:34');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (737480707,1,999442950,'KOSS',261,35.62,'2022-06-13 02:49:25','2022-06-13 02:49:33');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (935904254,1,695964930,'MDLZ',116,212.29,'2022-02-18 11:21:17','2022-02-18 11:21:25');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (353804975,1,229581407,'PHD',362,122.89,'2022-07-21 01:17:02','2022-07-21 01:17:04');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (570375897,1,051254898,'TAX',328,492.27,'2021-12-23 16:56:31','2021-12-23 16:56:31');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (780350820,1,504316197,'FPAY',432,376.55,'2022-07-29 12:57:05','2022-07-29 12:57:12');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (446144602,1,595669051,'SUNW',386,498.36,'2022-04-11 13:34:40','2022-04-11 13:34:48');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (148294901,1,724465022,'BSET',376,102.56,'2022-01-04 10:32:42','2022-01-04 10:32:45');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (955190154,1,755291445,'DLR^H',307,323.83,'2022-05-27 13:41:33','2022-05-27 13:41:36');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (330673734,1,571423123,'LBTYB',376,234.65,'2022-06-25 08:10:41','2022-06-25 08:10:42');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (431561345,1,654784427,'LEE',382,126.58,'2022-02-23 15:10:21','2022-02-23 15:10:28');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (848897728,1,287455122,'SPOK',398,286.79,'2022-03-14 06:04:15','2022-03-14 06:04:25');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (681321644,1,911484540,'VTIP',391,425.63,'2022-09-16 15:11:11','2022-09-16 15:11:18');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (275455087,1,044837238,'BTE',279,414.66,'2022-10-02 06:14:28','2022-10-02 06:14:32');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (078954099,1,550650228,'ONSIW',308,372.99,'2022-01-31 04:33:10','2022-01-31 04:33:11');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (297022287,1,864443046,'BLW',266,253.98,'2022-07-26 14:44:46','2022-07-26 14:44:51');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (805447896,1,938842398,'EMD',129,77.32,'2022-04-02 12:40:19','2022-04-02 12:40:24');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (456977863,1,095862058,'WASH',129,354.09,'2022-08-12 06:07:09','2022-08-12 06:07:16');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (951684217,1,326142432,'FITS',269,372.05,'2022-01-29 20:20:05','2022-01-29 20:20:11');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (120511505,1,708353551,'ATHM',372,340.17,'2022-09-26 22:54:33','2022-09-26 22:54:41');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (664176434,1,673161765,'TLND',390,101.21,'2022-07-01 15:14:06','2022-07-01 15:14:06');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (215242756,1,630469362,'KGC',123,101.31,'2022-01-12 12:44:27','2022-01-12 12:44:27');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (585414982,1,378409425,'APA',370,296.96,'2022-03-26 08:20:56','2022-03-26 08:21:01');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (187582319,1,342919060,'PMT^A',310,345.48,'2022-09-07 02:53:48','2022-09-07 02:53:50');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (879072917,1,995977002,'APB',333,390.43,'2022-07-05 12:37:59','2022-07-05 12:37:59');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (449234702,1,999181415,'DHIL',233,105.66,'2021-12-07 17:16:46','2021-12-07 17:16:56');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (408842067,1,989743527,'UMPQ',466,38.45,'2022-05-12 23:11:44','2022-05-12 23:11:47');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (287693836,1,356657976,'RNR^C',317,80.3,'2022-02-06 23:46:45','2022-02-06 23:46:54');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (633876529,1,966040820,'AMZN',368,120.68,'2022-08-13 05:19:50','2022-08-13 05:19:59');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (301406183,1,928140910,'PRE^G',423,182.52,'2022-01-29 18:16:46','2022-01-29 18:16:48');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (400656730,1,582022405,'NGLS^A',203,380.53,'2022-02-11 00:16:24','2022-02-11 00:16:28');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (500769464,1,602711048,'PTIE',267,213.77,'2022-02-10 03:58:06','2022-02-10 03:58:06');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (248533448,1,511744228,'VIVE',156,235.48,'2022-11-30 12:24:57','2022-11-30 12:25:00');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (254590773,1,169290390,'SHOO',309,366.01,'2022-10-02 15:37:13','2022-10-02 15:37:20');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (874863343,1,683047462,'VTTI',312,251.27,'2022-11-01 03:58:25','2022-11-01 03:58:33');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (890346130,1,296799067,'GS^D',480,488.24,'2022-06-23 07:19:01','2022-06-23 07:19:06');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (596761993,1,083297601,'CSGP',422,369.5,'2022-11-22 13:31:09','2022-11-22 13:31:14');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (913426527,1,006304966,'TK',287,356.13,'2022-05-30 15:14:08','2022-05-30 15:14:09');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (159460876,1,992754499,'TRMK',326,203.46,'2021-12-24 03:45:08','2021-12-24 03:45:18');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (509693238,1,714791398,'AMGP',287,462.68,'2022-04-02 18:40:38','2022-04-02 18:40:41');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (164590310,1,594752856,'AHGP',398,131.05,'2022-06-06 22:54:31','2022-06-06 22:54:41');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (160033185,1,023767170,'CRBP',219,238.04,'2022-07-24 19:21:47','2022-07-24 19:21:53');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (131925739,1,488906694,'GS^J',362,420.27,'2022-04-25 19:19:36','2022-04-25 19:19:40');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (629595859,1,122164559,'NWL',478,212.52,'2022-04-06 13:33:58','2022-04-06 13:34:03');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (449729485,1,771132787,'SFUN',183,388.77,'2022-03-20 07:01:07','2022-03-20 07:01:08');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (270382517,1,517518405,'BMO',276,284.14,'2021-12-23 05:18:06','2021-12-23 05:18:13');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (483871728,1,249223899,'IBCP',248,178.45,'2022-03-04 12:27:07','2022-03-04 12:27:09');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (586076139,1,286947701,'RM',197,31.96,'2022-04-06 03:01:28','2022-04-06 03:01:34');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (417646441,1,717285490,'MMU',275,294.76,'2022-07-15 07:16:28','2022-07-15 07:16:35');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (050777197,1,868094915,'AKBA',251,360.75,'2022-01-25 16:07:20','2022-01-25 16:07:29');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (978433443,1,479193672,'TICC',163,134.11,'2022-07-04 14:58:15','2022-07-04 14:58:17');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (058856095,1,078371468,'DVD',488,281.87,'2022-11-17 14:50:10','2022-11-17 14:50:16');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (749795583,1,624536216,'MDIV',199,301.48,'2022-03-27 21:51:39','2022-03-27 21:51:43');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (079834377,1,038769285,'OC',248,432.06,'2022-03-29 11:43:50','2022-03-29 11:43:57');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (934313888,1,894607348,'GSBD',242,365.93,'2021-12-22 12:22:04','2021-12-22 12:22:05');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (788265066,1,390456947,'LVHD',212,489.53,'2022-10-30 07:51:47','2022-10-30 07:51:47');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (914593303,1,887063292,'NWBI',417,431.66,'2022-11-30 13:04:37','2022-11-30 13:04:40');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (384800609,1,332263631,'STWD',253,43.12,'2022-10-02 00:05:43','2022-10-02 00:05:43');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (504798721,1,557937120,'KDMN',381,118.61,'2022-06-18 06:14:07','2022-06-18 06:14:07');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (566374393,1,418823469,'WLK',286,158.17,'2022-06-01 17:39:51','2022-06-01 17:40:01');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (773888868,1,847331620,'PTI',361,45.92,'2022-10-13 12:20:35','2022-10-13 12:20:43');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (744671524,1,376975473,'CLLS',235,130.36,'2022-09-17 02:21:34','2022-09-17 02:21:43');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (672149598,1,489476826,'UGLD',264,213.03,'2022-07-18 03:50:08','2022-07-18 03:50:16');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (684613401,1,161079714,'CLNS^H',495,76.35,'2022-02-07 07:23:46','2022-02-07 07:23:52');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (887391177,1,474989301,'GFN',400,362.69,'2022-03-23 07:55:48','2022-03-23 07:55:52');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (593787145,1,499287568,'CRVL',220,359.62,'2022-08-11 05:44:45','2022-08-11 05:44:52');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (652757717,1,382443675,'RESN',330,411.93,'2022-09-11 16:32:11','2022-09-11 16:32:13');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (876986694,1,734000434,'ACRX',430,250.83,'2022-01-14 04:25:37','2022-01-14 04:25:44');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (762279068,1,089251970,'TXT',374,156.22,'2022-08-27 02:24:44','2022-08-27 02:24:44');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (079016036,1,254513008,'FNCX',121,417.31,'2022-06-09 20:24:16','2022-06-09 20:24:16');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (195950954,1,994555123,'DRWI',191,337.98,'2022-06-07 23:45:52','2022-06-07 23:46:01');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (693495413,1,461403256,'RUSHA',432,251.63,'2022-05-17 23:06:08','2022-05-17 23:06:18');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (348964389,1,213188754,'EACQ',314,309.88,'2022-09-01 19:51:17','2022-09-01 19:51:22');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (927124633,1,494611732,'PCOM',279,124.91,'2022-06-05 12:29:58','2022-06-05 12:30:06');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (852250699,1,978457108,'SCACU',255,7.33,'2022-09-15 15:52:23','2022-09-15 15:52:30');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (698457482,1,185231422,'XOXO',403,460.54,'2022-04-14 13:46:13','2022-04-14 13:46:16');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (700914116,1,480161486,'QUMU',153,275.64,'2022-01-10 06:07:53','2022-01-10 06:08:00');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (831166050,1,023775564,'PEIX',109,183.65,'2022-05-02 22:06:18','2022-05-02 22:06:23');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (691862907,1,590592162,'COL',468,268.12,'2021-12-20 11:08:01','2021-12-20 11:08:04');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (426263116,1,637473567,'TDC',413,14.25,'2022-03-17 13:48:57','2022-03-17 13:49:00');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (495404063,1,254200118,'SR',459,127.62,'2022-05-30 01:43:25','2022-05-30 01:43:30');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (559074637,1,750463862,'HIX',197,7.95,'2022-02-01 08:00:15','2022-02-01 08:00:15');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (674684629,1,335095359,'SOR',254,178.05,'2022-03-17 14:32:28','2022-03-17 14:32:38');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (825361373,1,128122731,'SYNC',475,347.94,'2022-03-06 21:52:41','2022-03-06 21:52:47');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (079920109,1,282953378,'PTC',352,207.71,'2021-12-31 09:35:37','2021-12-31 09:35:39');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (063011320,1,156658556,'PDLI',334,340.06,'2022-07-22 22:08:30','2022-07-22 22:08:31');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (938443261,1,230039270,'DSWL',277,74.6,'2022-02-11 04:42:53','2022-02-11 04:42:55');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (605915051,1,025987240,'CPB',439,100.16,'2022-07-26 09:08:23','2022-07-26 09:08:30');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (152314317,1,847054847,'MDSO',141,187.31,'2022-02-17 22:26:26','2022-02-17 22:26:31');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (090384555,1,594965059,'NYMTP',478,352.55,'2022-06-09 15:49:41','2022-06-09 15:49:48');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (176871936,1,851919263,'GBL',369,85.51,'2022-09-22 05:34:29','2022-09-22 05:34:35');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (956072868,1,560634525,'CHS',444,324.53,'2022-02-12 22:28:54','2022-02-12 22:29:02');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (053446702,1,363242941,'GOGL',224,261.43,'2022-07-21 02:30:47','2022-07-21 02:30:55');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (963697428,1,797589909,'JBSS',247,317.2,'2022-02-11 02:08:33','2022-02-11 02:08:42');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (109331956,1,008819798,'TGI',285,482.89,'2022-02-26 05:22:50','2022-02-26 05:22:52');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (748025239,1,240899073,'FLC',433,210.78,'2022-01-28 12:30:45','2022-01-28 12:30:54');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (117035715,1,582522518,'ASV',173,334.17,'2022-07-11 09:11:07','2022-07-11 09:11:09');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (792614410,1,125875572,'SBGI',406,5.16,'2022-10-15 10:03:51','2022-10-15 10:04:01');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (697756262,1,542027005,'NVMI',203,396.64,'2022-09-02 04:20:56','2022-09-02 04:20:59');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (228954017,1,244628470,'KMT',127,318.7,'2022-03-27 14:00:53','2022-03-27 14:00:58');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (136603195,1,550575750,'STWD',209,406.39,'2022-07-21 07:08:39','2022-07-21 07:08:46');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (980429343,1,587524774,'AIV^A',171,249.69,'2022-03-05 16:16:49','2022-03-05 16:16:58');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (612298213,1,296328766,'ABB',424,95.87,'2022-06-15 00:37:20','2022-06-15 00:37:26');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (789093872,1,712826496,'LAMR',471,225.03,'2022-11-10 08:02:47','2022-11-10 08:02:48');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (022062250,1,496707528,'ENDP',204,481.69,'2022-06-18 22:19:11','2022-06-18 22:19:15');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (832572992,1,807071871,'ESGG',370,461.54,'2022-03-28 15:00:41','2022-03-28 15:00:41');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (342898332,1,387343439,'AXTA',256,466.0,'2022-06-13 21:49:32','2022-06-13 21:49:35');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (139306703,1,366846045,'EFAS',342,217.34,'2022-10-13 02:46:20','2022-10-13 02:46:26');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (047590116,1,088451452,'DRWI',142,42.48,'2022-04-16 09:59:24','2022-04-16 09:59:24');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (735023894,1,856836482,'DBD',372,298.74,'2021-12-10 13:00:38','2021-12-10 13:00:38');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (000954417,1,452572457,'CVO',490,325.09,'2022-02-28 07:49:31','2022-02-28 07:49:41');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (871285608,1,286819083,'AMCN',246,161.49,'2022-09-19 08:15:10','2022-09-19 08:15:13');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (879445846,1,948705948,'VRTV',417,374.38,'2022-05-17 07:21:01','2022-05-17 07:21:04');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (290742914,1,842398595,'CLR',456,464.52,'2022-08-27 10:09:52','2022-08-27 10:09:55');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (149624677,1,581392627,'MGYR',334,381.79,'2022-03-07 04:14:25','2022-03-07 04:14:32');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (234169858,1,026853316,'VSH',489,271.57,'2022-03-10 09:19:53','2022-03-10 09:19:58');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (609086869,1,249439234,'MFC',487,488.67,'2022-01-23 02:22:03','2022-01-23 02:22:03');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (367608495,1,136795122,'LPSN',429,189.91,'2022-10-30 21:04:53','2022-10-30 21:04:54');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (680475814,1,320555851,'RTIX',460,344.89,'2022-09-27 02:51:42','2022-09-27 02:51:48');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (709898048,1,704570833,'JMPB',367,294.09,'2022-07-08 22:35:42','2022-07-08 22:35:50');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (930783072,1,196894924,'FOMX',160,106.19,'2022-05-07 07:30:00','2022-05-07 07:30:04');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (617723660,1,283155297,'TBNK',116,402.27,'2022-11-05 04:09:42','2022-11-05 04:09:49');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (360873608,1,915453961,'TWNKW',428,304.27,'2022-05-17 09:37:46','2022-05-17 09:37:50');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (724456243,1,998325118,'ATHM',-237,97.7382248423,'2022-08-31 01:49:55','2022-08-31 01:50:04');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (507153238,1,300585899,'COF^D',-226,65.2055586951,'2022-04-30 15:17:39','2022-04-30 15:17:47');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (183585804,1,608530551,'GAB',-270,398.8443480387,'2022-02-11 05:51:52','2022-02-11 05:52:01');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (647630435,1,375500853,'CRS',-144,291.7637006984,'2022-08-25 00:08:03','2022-08-25 00:08:06');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (928542586,1,274391853,'EQGP',-337,248.9893343571,'2022-08-05 00:08:23','2022-08-05 00:08:31');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (348057928,1,820638991,'VOD',-491,389.2091465655,'2022-05-02 17:43:22','2022-05-02 17:43:28');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (021277373,1,872630201,'RDC',-217,359.0828183404,'2022-04-11 08:00:46','2022-04-11 08:00:48');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (839879164,1,956707872,'PFGC',-110,77.2922935735,'2022-12-02 19:04:51','2022-12-02 19:04:58');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (302077615,1,923792033,'RLGY',-289,425.2354446394,'2022-05-07 07:11:16','2022-05-07 07:11:22');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (099321410,1,250415049,'RVLT',-240,212.481097052,'2022-06-04 16:48:54','2022-06-04 16:49:02');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (960013745,1,459178493,'BOSC',-473,260.3550384197,'2022-02-02 04:03:21','2022-02-02 04:03:30');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (654636774,1,329626266,'NUROW',-277,268.4647675784,'2022-08-25 04:59:38','2022-08-25 04:59:40');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (742835107,1,466560851,'NUTR',-414,347.2725793232,'2022-11-25 23:06:03','2022-11-25 23:06:04');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (260149288,1,965514288,'PHO',-150,443.7532531103,'2022-03-17 15:40:18','2022-03-17 15:40:25');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (148216098,1,912632634,'HVT.A',-310,48.8065976175,'2022-09-16 06:37:34','2022-09-16 06:37:37');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (182220833,1,750566235,'FH',-275,180.1858823175,'2022-11-12 00:38:25','2022-11-12 00:38:34');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (497835833,1,491606976,'TSRI',-444,156.0136349998,'2022-11-07 02:48:02','2022-11-07 02:48:05');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (126099012,1,116185127,'JPM^G',-118,356.8446906086,'2022-10-24 15:14:40','2022-10-24 15:14:42');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (437807662,1,427437569,'RMT',-121,47.4215904532,'2022-04-26 02:03:24','2022-04-26 02:03:32');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (167807285,1,162906379,'NTZ',-134,189.7036037505,'2022-02-18 08:05:22','2022-02-18 08:05:27');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (848047362,1,323945588,'LHO^J',-416,144.194377196,'2022-10-03 13:12:18','2022-10-03 13:12:28');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (730789157,1,004555319,'EMCF',-425,281.5389499972,'2022-08-22 08:48:23','2022-08-22 08:48:30');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (764967809,1,739016238,'PBCTP',-355,69.2001924624,'2022-09-16 18:52:31','2022-09-16 18:52:34');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (874012278,1,334604287,'GLMD',-478,36.7349714077,'2022-09-24 03:46:07','2022-09-24 03:46:11');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (109310601,1,614004145,'SPE^B',-293,444.7915580329,'2022-02-02 23:07:10','2022-02-02 23:07:15');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (812846866,1,114541558,'PIO',-495,398.0886001339,'2022-05-18 15:34:03','2022-05-18 15:34:13');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (452674361,1,603296925,'UNM',-401,128.9048816855,'2022-02-17 00:38:44','2022-02-17 00:38:47');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (995212776,1,060637430,'CSL',-139,313.9186988836,'2022-02-03 09:48:40','2022-02-03 09:48:43');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (409983840,1,319716523,'SUNS',-408,365.5103330917,'2022-11-05 13:29:25','2022-11-05 13:29:26');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (483801610,1,105425552,'ETX',-483,308.7719628818,'2022-06-10 08:13:29','2022-06-10 08:13:39');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (273891946,1,412606721,'MFSF',-372,582.2236445523,'2022-07-19 14:10:08','2022-07-19 14:10:14');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (122157457,1,668865077,'HTHT',-227,108.2219415377,'2022-05-01 07:43:23','2022-05-01 07:43:29');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (402409845,1,667385692,'MTGEP',-480,285.2888139772,'2022-03-30 11:46:48','2022-03-30 11:46:57');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (304551589,1,809629504,'AUPH',-349,46.4525693699,'2022-04-25 20:30:40','2022-04-25 20:30:41');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (283681522,1,826060268,'SPAN',-482,273.6093752487,'2022-11-21 16:58:41','2022-11-21 16:58:49');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (938360160,1,221275029,'TRIL',-341,140.6327068,'2022-01-29 19:01:25','2022-01-29 19:01:28');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (121810008,1,030490782,'GEF.B',-307,456.8246677433,'2022-01-29 00:39:54','2022-01-29 00:39:58');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (979282486,1,021904830,'ANCX',-251,179.0787786456,'2022-07-10 02:07:25','2022-07-10 02:07:34');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (479034996,1,429373670,'ISBC',-163,326.4939815145,'2022-09-19 06:39:17','2022-09-19 06:39:26');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (409582907,1,249797132,'WLB',-168,401.4536937036,'2022-01-24 14:23:36','2022-01-24 14:23:46');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (366857703,1,769321991,'JEC',-448,155.3988759461,'2022-01-06 16:26:34','2022-01-06 16:26:37');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (662402422,1,499684733,'BGE^B',-383,458.4824028209,'2022-06-22 23:56:09','2022-06-22 23:56:13');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (805805443,1,082927608,'LALT',-259,510.1691392942,'2022-10-04 02:26:15','2022-10-04 02:26:22');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (119226955,1,512536957,'ADRU',-227,4.177368585,'2022-08-11 04:02:33','2022-08-11 04:02:39');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (462215404,1,493467432,'LJPC',-432,664.051271806,'2022-04-18 04:18:55','2022-04-18 04:18:56');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (791040800,1,142660041,'NI',-394,165.2906993241,'2022-10-08 08:02:33','2022-10-08 08:02:43');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (325899375,1,628189671,'UBFO',-135,477.3284885064,'2022-07-15 09:32:38','2022-07-15 09:32:43');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (230495712,1,331633006,'ADHD',-301,106.8160287467,'2022-06-12 05:19:44','2022-06-12 05:19:52');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (995306119,1,516575097,'MRK',-174,125.0901652131,'2022-09-28 12:44:37','2022-09-28 12:44:44');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (249449334,1,318906343,'ADBE',-394,255.7847367819,'2022-01-05 16:37:15','2022-01-05 16:37:25');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (677178570,1,985885601,'WBKC',-452,184.6058544639,'2022-04-21 02:36:05','2022-04-21 02:36:12');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (992439491,1,918081156,'L',-291,351.0780997623,'2022-11-22 16:17:45','2022-11-22 16:17:49');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (160798153,1,778764675,'WABC',-273,327.1828089976,'2022-02-07 19:15:31','2022-02-07 19:15:34');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (304851221,1,026355223,'JE',-423,419.3837800881,'2022-03-21 22:34:16','2022-03-21 22:34:22');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (163906167,1,592084726,'BXE',-397,461.0463443985,'2022-09-01 01:06:37','2022-09-01 01:06:40');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (531562439,1,942431792,'TWNKW',-354,384.6623535033,'2022-05-30 17:18:36','2022-05-30 17:18:42');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (201073864,1,460130244,'LGF.B',-265,226.8246381305,'2022-04-13 03:46:20','2022-04-13 03:46:29');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (191788980,1,262348555,'BICK',-280,431.5215820718,'2022-03-27 01:54:12','2022-03-27 01:54:22');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (817636124,1,282079415,'NEE^C',-392,389.3809917959,'2022-06-24 11:57:43','2022-06-24 11:57:52');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (203621951,1,324101738,'QQQX',-423,85.732975301,'2022-03-14 04:21:11','2022-03-14 04:21:14');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (491937855,1,676609785,'SMIT',-131,279.6505271685,'2022-05-16 00:11:24','2022-05-16 00:11:34');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (072102377,1,580798972,'LBIO',-328,354.0874537313,'2022-06-29 18:25:47','2022-06-29 18:25:56');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (260939748,1,311914677,'CACC',-111,257.1676528397,'2022-09-25 15:09:56','2022-09-25 15:10:03');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (550759490,1,657301532,'PACB',-468,111.9479962685,'2022-10-22 08:17:34','2022-10-22 08:17:42');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (685561304,1,275156034,'BSPM',-272,38.5934121003,'2022-03-12 10:05:56','2022-03-12 10:06:04');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (107857538,1,909594814,'MEOH',-285,683.8550398701,'2022-08-17 15:20:44','2022-08-17 15:20:49');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (474023503,1,109908460,'SIMO',-252,475.7370969238,'2022-03-24 18:28:00','2022-03-24 18:28:04');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (461783207,1,532339969,'PTR',-109,163.9810885549,'2022-07-27 05:46:52','2022-07-27 05:46:59');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (282712697,1,228065620,'HWBK',-246,362.260412896,'2022-09-25 00:55:54','2022-09-25 00:55:55');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (216743366,1,282075074,'ADXS',-469,198.1747116266,'2022-08-05 00:48:23','2022-08-05 00:48:27');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (177543067,1,998104368,'XRX',-235,43.7710693235,'2022-05-22 06:40:57','2022-05-22 06:41:06');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (426697200,1,662214680,'GBLIL',-119,237.5375476724,'2022-06-10 15:25:01','2022-06-10 15:25:06');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (983537263,1,076255005,'KNL',-340,446.5021952629,'2022-09-10 18:16:00','2022-09-10 18:16:10');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (917443164,1,834471688,'YUME',-357,291.7682137072,'2022-05-24 22:30:05','2022-05-24 22:30:11');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (556112380,1,239847666,'PTIE',-498,21.6162592641,'2022-11-11 15:04:44','2022-11-11 15:04:54');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (279873699,1,818775600,'LRAD',-134,292.8616822517,'2022-10-06 11:53:35','2022-10-06 11:53:43');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (574969236,1,550413487,'WAFDW',-424,150.7390119049,'2022-04-30 02:47:23','2022-04-30 02:47:24');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (470606070,1,713394497,'AVT',-216,46.4626078,'2022-05-26 19:35:39','2022-05-26 19:35:44');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (377209044,1,726957289,'GAIA',-220,407.4521846668,'2022-05-19 16:54:13','2022-05-19 16:54:14');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (616648107,1,574906844,'OPP',-310,163.4569109393,'2022-01-03 09:02:26','2022-01-03 09:02:34');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (935781361,1,311161537,'COLL',-107,69.8187844401,'2022-04-13 09:39:08','2022-04-13 09:39:10');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (861108654,1,325819132,'WYN',-465,167.5210499085,'2022-01-20 11:56:47','2022-01-20 11:56:55');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (133677419,1,746446204,'SNP',-232,225.1991599493,'2022-01-25 13:08:44','2022-01-25 13:08:51');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (865424493,1,977263673,'KRC^H',-163,341.1018119983,'2022-11-07 22:51:49','2022-11-07 22:51:53');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (142356337,1,743118607,'MNST',-450,328.8888519042,'2022-08-29 18:25:51','2022-08-29 18:25:59');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (500593316,1,675416933,'WLH',-199,448.9196588196,'2022-05-27 05:51:22','2022-05-27 05:51:28');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (854918224,1,397180433,'ALX',-346,592.7327584641,'2022-05-27 18:16:45','2022-05-27 18:16:53');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (875601810,1,929555760,'LBIX',-183,270.7379173701,'2022-07-31 20:48:41','2022-07-31 20:48:45');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (375153720,1,139480550,'BSAC',-166,192.1554617523,'2022-05-31 08:20:37','2022-05-31 08:20:46');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (091063616,1,171327069,'TPVZ',-288,503.6193960474,'2022-04-21 10:48:34','2022-04-21 10:48:35');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (562514354,1,952756584,'CHMI',-372,359.539187292,'2022-02-18 09:19:30','2022-02-18 09:19:38');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (014754413,1,044409115,'WAGE',-407,447.5675880229,'2022-10-01 05:05:00','2022-10-01 05:05:09');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (800138094,1,835059712,'NGHCN',-274,162.9160678945,'2022-05-11 04:35:47','2022-05-11 04:35:51');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (510273407,1,693880970,'IPWR',-318,294.3221309378,'2022-07-22 12:44:11','2022-07-22 12:44:18');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (198439266,1,660458440,'PBR.A',-375,210.2870231815,'2022-01-13 15:16:23','2022-01-13 15:16:33');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (987473614,1,381145924,'NEWM',-291,516.7756771068,'2022-04-29 16:27:15','2022-04-29 16:27:25');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (565379071,1,476704648,'OCUL',-193,160.2151622276,'2022-06-19 03:16:40','2022-06-19 03:16:48');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (052905032,1,004965884,'LORL',-203,63.6409253308,'2022-11-29 14:00:55','2022-11-29 14:00:56');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (514402369,1,739502250,'MON',-358,108.8709956837,'2022-08-27 03:10:31','2022-08-27 03:10:41');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (626203634,1,796284004,'CWBC',-210,191.4297817016,'2022-05-05 23:08:48','2022-05-05 23:08:51');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (074897657,1,231075256,'KWR',-426,259.2188000976,'2022-10-01 12:36:21','2022-10-01 12:36:26');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (645581274,1,642506826,'ECR',-357,294.6875480757,'2022-04-29 21:57:14','2022-04-29 21:57:17');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (609088444,1,753593819,'CELH',-113,93.1721847218,'2022-03-31 17:07:41','2022-03-31 17:07:45');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (920868365,1,921476346,'WYNN',-365,260.4319346137,'2022-06-24 05:35:15','2022-06-24 05:35:17');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (756224184,1,017682620,'SQM',-215,247.4503572227,'2022-01-13 13:41:38','2022-01-13 13:41:42');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (336585348,1,561877915,'CTB',-229,171.2289812382,'2022-06-18 21:23:44','2022-06-18 21:23:47');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (464751780,1,761782348,'FNFV',-311,317.1280746163,'2022-05-30 05:32:37','2022-05-30 05:32:41');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (508331164,1,034921263,'PDLI',-147,96.6756431511,'2022-01-21 10:34:33','2022-01-21 10:34:41');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (702362628,1,798894307,'EMES',-297,228.8018601965,'2022-04-26 17:51:13','2022-04-26 17:51:18');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (521018394,1,708430465,'WAB',-378,33.272899526,'2022-07-15 17:30:45','2022-07-15 17:30:53');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (201371021,1,647852592,'HRS',-315,77.2513372775,'2022-12-01 00:31:08','2022-12-01 00:31:10');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (276277158,1,907014647,'KREF',-419,324.9247111813,'2022-07-11 07:54:07','2022-07-11 07:54:14');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (333021993,1,747615399,'JP',-444,316.0261868056,'2022-05-18 10:12:33','2022-05-18 10:12:41');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (642637211,1,715809122,'RRC',-112,21.9283246163,'2022-05-14 05:14:43','2022-05-14 05:14:44');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (772851728,1,167207503,'BLH',-177,103.4551353548,'2022-09-20 15:23:49','2022-09-20 15:23:54');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (841112478,1,646937032,'WCST',-415,115.9764913644,'2022-11-14 00:07:06','2022-11-14 00:07:14');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (166622260,1,539056113,'B',-212,118.2117573769,'2022-04-09 07:29:43','2022-04-09 07:29:46');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (882583362,1,731317912,'ARW',-175,446.7111149725,'2022-02-26 21:51:12','2022-02-26 21:51:16');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (615362540,1,074780072,'AWP',-123,300.516987011,'2022-09-18 00:53:42','2022-09-18 00:53:44');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (771874962,1,130854604,'NCR',-479,371.7569380403,'2022-02-24 09:09:28','2022-02-24 09:09:38');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (508727076,1,412202357,'GHC',-243,75.0881515794,'2022-05-25 09:58:26','2022-05-25 09:58:34');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (791513205,1,754086541,'VLRS',-371,188.7604590123,'2022-01-28 21:06:28','2022-01-28 21:06:29');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (966180938,1,385740146,'CYD',-484,280.552675274,'2022-01-01 02:04:44','2022-01-01 02:04:53');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (932256252,1,206054292,'FTAI',-168,141.8821404838,'2022-06-27 04:31:55','2022-06-27 04:31:59');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (682637729,1,288431752,'ARAY',-103,124.0075934443,'2022-03-15 19:26:10','2022-03-15 19:26:12');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (271088133,1,557782593,'ORI',-356,442.9355700099,'2022-08-04 05:44:49','2022-08-04 05:44:51');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (778165911,1,119429397,'CPN',-100,188.2970092933,'2022-08-17 08:34:14','2022-08-17 08:34:15');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (269109954,1,848733680,'CYCC',-457,294.9129507928,'2022-10-23 09:16:22','2022-10-23 09:16:28');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (802210978,1,999442950,'KOSS',-261,41.1616448568,'2022-09-14 16:27:44','2022-09-14 16:27:49');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (406374428,1,695964930,'MDLZ',-116,231.4001146338,'2022-01-14 08:52:06','2022-01-14 08:52:10');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (686952450,1,229581407,'PHD',-362,145.1050376928,'2022-11-05 15:06:16','2022-11-05 15:06:19');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (000715132,1,051254898,'TAX',-328,521.6912281962,'2022-03-10 16:29:10','2022-03-10 16:29:17');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (985439466,1,504316197,'FPAY',-432,552.0327797345,'2022-05-01 16:39:34','2022-05-01 16:39:35');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (241724503,1,595669051,'SUNW',-386,554.6120007655,'2022-01-24 15:47:27','2022-01-24 15:47:32');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (413479797,1,724465022,'BSET',-376,124.6877519901,'2022-05-27 16:27:34','2022-05-27 16:27:36');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (770526944,1,755291445,'DLR^H',-307,243.4106369392,'2022-08-15 07:48:36','2022-08-15 07:48:39');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (936685864,1,571423123,'LBTYB',-376,267.6482088433,'2022-06-19 21:54:17','2022-06-19 21:54:22');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (460429384,1,654784427,'LEE',-382,128.0958457446,'2022-10-25 02:23:26','2022-10-25 02:23:31');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (146858311,1,287455122,'SPOK',-398,214.9306737975,'2022-08-31 04:36:36','2022-08-31 04:36:46');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (959401704,1,911484540,'VTIP',-391,631.1283748362,'2022-10-03 11:44:35','2022-10-03 11:44:40');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (458290498,1,044837238,'BTE',-279,395.3816967338,'2022-05-13 15:22:04','2022-05-13 15:22:11');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (053435003,1,550650228,'ONSIW',-308,365.3433568869,'2022-04-03 05:19:28','2022-04-03 05:19:34');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (658844524,1,864443046,'BLW',-266,197.9773869906,'2022-10-10 03:29:07','2022-10-10 03:29:12');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (360953465,1,938842398,'EMD',-129,74.8487468526,'2022-01-31 19:00:15','2022-01-31 19:00:25');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (027887192,1,095862058,'WASH',-129,417.9484032288,'2022-01-05 23:07:40','2022-01-05 23:07:45');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (710190287,1,326142432,'FITS',-269,368.910882007,'2022-08-03 09:11:45','2022-08-03 09:11:51');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (392231626,1,708353551,'ATHM',-372,464.2007435073,'2022-02-17 06:22:47','2022-02-17 06:22:52');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (264626702,1,673161765,'TLND',-390,84.0769202387,'2022-08-15 19:56:40','2022-08-15 19:56:47');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (242847357,1,630469362,'KGC',-123,105.3954102091,'2022-02-20 09:10:03','2022-02-20 09:10:07');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (888695300,1,378409425,'APA',-370,329.7159415913,'2022-02-26 02:11:55','2022-02-26 02:12:02');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (309849179,1,342919060,'PMT^A',-310,349.595613046,'2022-04-30 21:44:58','2022-04-30 21:45:03');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (549492207,1,995977002,'APB',-333,542.620412943,'2022-10-11 14:48:38','2022-10-11 14:48:45');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (516624569,1,999181415,'DHIL',-233,123.2867625361,'2022-04-21 18:02:00','2022-04-21 18:02:01');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (401030976,1,989743527,'UMPQ',-466,56.9350421467,'2022-11-01 19:29:01','2022-11-01 19:29:04');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (682337916,1,356657976,'RNR^C',-317,88.3385547326,'2022-07-22 10:09:18','2022-07-22 10:09:21');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (381759911,1,966040820,'AMZN',-368,114.5925945947,'2022-02-18 12:57:49','2022-02-18 12:57:58');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (824417939,1,928140910,'PRE^G',-423,139.4596981009,'2022-11-09 21:46:57','2022-11-09 21:47:02');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (574242508,1,582022405,'NGLS^A',-203,527.7775133664,'2022-08-13 18:58:49','2022-08-13 18:58:53');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (959115159,1,602711048,'PTIE',-267,260.8021620311,'2022-06-19 19:13:54','2022-06-19 19:13:59');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (139471026,1,511744228,'VIVE',-156,276.8588903573,'2022-08-26 16:58:07','2022-08-26 16:58:12');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (156617214,1,169290390,'SHOO',-309,459.4393395404,'2022-07-28 10:50:55','2022-07-28 10:50:58');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (648032792,1,683047462,'VTTI',-312,374.8419738533,'2022-01-12 02:49:57','2022-01-12 02:49:58');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (281089308,1,296799067,'GS^D',-480,525.6651652136,'2022-09-01 01:29:22','2022-09-01 01:29:27');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (723813260,1,083297601,'CSGP',-422,410.9928698459,'2022-08-06 11:45:46','2022-08-06 11:45:54');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (650710383,1,006304966,'TK',-287,443.738380089,'2022-03-15 02:03:13','2022-03-15 02:03:23');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (937573406,1,992754499,'TRMK',-326,157.5331957202,'2022-06-16 17:27:47','2022-06-16 17:27:52');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (419592851,1,714791398,'AMGP',-287,336.2075431878,'2022-05-02 05:06:24','2022-05-02 05:06:26');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (093618185,1,594752856,'AHGP',-398,136.827137981,'2022-06-19 00:23:33','2022-06-19 00:23:42');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (152274897,1,023767170,'CRBP',-219,242.0549480509,'2022-02-12 06:12:52','2022-02-12 06:12:57');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (895835870,1,488906694,'GS^J',-362,431.2337561621,'2022-04-04 05:52:10','2022-04-04 05:52:20');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (066241638,1,122164559,'NWL',-478,180.1737130617,'2022-09-25 03:51:25','2022-09-25 03:51:26');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (070264479,1,771132787,'SFUN',-183,418.7484831715,'2022-10-08 04:40:59','2022-10-08 04:41:05');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (140529075,1,517518405,'BMO',-276,220.1154913575,'2022-02-27 02:30:10','2022-02-27 02:30:18');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (418652386,1,249223899,'IBCP',-248,137.3585029829,'2022-04-09 23:08:58','2022-04-09 23:09:04');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (187851663,1,286947701,'RM',-197,44.3717691067,'2022-06-11 22:58:19','2022-06-11 22:58:21');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (517110331,1,717285490,'MMU',-275,246.0708557041,'2022-06-27 09:13:59','2022-06-27 09:14:08');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (615488431,1,868094915,'AKBA',-251,514.0906695011,'2022-05-10 11:18:06','2022-05-10 11:18:14');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (689501012,1,479193672,'TICC',-163,140.5393751152,'2022-04-26 07:51:07','2022-04-26 07:51:09');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (572404106,1,078371468,'DVD',-488,370.2533410532,'2022-05-19 21:09:10','2022-05-19 21:09:15');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (085105232,1,624536216,'MDIV',-199,348.98633739,'2022-06-26 09:42:06','2022-06-26 09:42:09');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (543971473,1,038769285,'OC',-248,473.7968844232,'2022-08-10 06:27:59','2022-08-10 06:28:07');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (669565913,1,894607348,'GSBD',-242,259.7169415644,'2022-01-05 23:45:18','2022-01-05 23:45:23');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (955408114,1,390456947,'LVHD',-212,552.4775010073,'2022-09-23 09:40:24','2022-09-23 09:40:34');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (684760435,1,887063292,'NWBI',-417,476.432893774,'2022-05-22 15:41:48','2022-05-22 15:41:56');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (397335661,1,332263631,'STWD',-253,46.0519221222,'2022-03-21 23:33:44','2022-03-21 23:33:46');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (719474720,1,557937120,'KDMN',-381,137.398616766,'2022-10-01 09:30:48','2022-10-01 09:30:51');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (789361317,1,418823469,'WLK',-286,112.3351586412,'2022-10-23 22:01:47','2022-10-23 22:01:49');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (660417252,1,847331620,'PTI',-361,42.0379613971,'2022-10-28 22:08:43','2022-10-28 22:08:47');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (012218623,1,376975473,'CLLS',-235,161.3184553646,'2022-03-15 06:06:53','2022-03-15 06:06:58');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (375782114,1,489476826,'UGLD',-264,249.4384400685,'2022-07-07 00:53:43','2022-07-07 00:53:51');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (552836247,1,161079714,'CLNS^H',-495,70.4218259377,'2022-09-23 20:27:50','2022-09-23 20:27:58');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (924110946,1,474989301,'GFN',-400,478.0055831286,'2022-10-05 21:02:05','2022-10-05 21:02:06');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (513774858,1,499287568,'CRVL',-220,460.7339087585,'2022-06-15 22:57:25','2022-06-15 22:57:29');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (597198219,1,382443675,'RESN',-330,559.380588999,'2022-06-03 17:21:05','2022-06-03 17:21:08');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (433314685,1,734000434,'ACRX',-430,299.2029994844,'2022-07-14 19:51:48','2022-07-14 19:51:50');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (267376992,1,089251970,'TXT',-374,133.349858671,'2022-02-02 13:02:36','2022-02-02 13:02:44');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (213506402,1,254513008,'FNCX',-121,620.576694827,'2022-09-21 05:05:15','2022-09-21 05:05:19');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (216385849,1,994555123,'DRWI',-191,357.873758568,'2022-09-09 00:59:03','2022-09-09 00:59:04');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (737433513,1,461403256,'RUSHA',-432,270.6115418678,'2022-08-18 09:59:27','2022-08-18 09:59:35');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (504907046,1,213188754,'EACQ',-314,406.766933357,'2022-05-02 16:59:46','2022-05-02 16:59:55');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (289167340,1,494611732,'PCOM',-279,113.0884238711,'2022-08-05 05:50:48','2022-08-05 05:50:57');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (617118582,1,978457108,'SCACU',-255,9.175556376,'2022-08-01 20:01:42','2022-08-01 20:01:48');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (965382100,1,185231422,'XOXO',-403,666.4147705251,'2022-06-09 09:03:55','2022-06-09 09:04:03');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (338071648,1,480161486,'QUMU',-153,315.1990573199,'2022-09-23 06:24:48','2022-09-23 06:24:56');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (384775296,1,023775564,'PEIX',-109,241.959214966,'2022-03-09 20:31:53','2022-03-09 20:31:59');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (188155166,1,590592162,'COL',-468,208.9114403953,'2022-08-02 12:22:54','2022-08-02 12:22:57');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (214013122,1,637473567,'TDC',-413,12.2709345727,'2022-08-07 05:16:49','2022-08-07 05:16:54');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (787427046,1,254200118,'SR',-459,125.9719554681,'2022-09-29 21:33:59','2022-09-29 21:34:03');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (869022452,1,750463862,'HIX',-197,6.6439610394,'2022-07-06 04:52:18','2022-07-06 04:52:28');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (325357978,1,335095359,'SOR',-254,215.6369998537,'2022-06-02 00:44:55','2022-06-02 00:44:59');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (913756485,1,128122731,'SYNC',-475,353.2924957955,'2022-01-04 17:39:34','2022-01-04 17:39:35');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (900964756,1,282953378,'PTC',-352,167.1042083612,'2022-02-15 13:30:14','2022-02-15 13:30:21');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (158108418,1,156658556,'PDLI',-334,389.8792909943,'2022-01-31 09:10:03','2022-01-31 09:10:10');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (988682581,1,230039270,'DSWL',-277,56.3827636422,'2022-04-12 15:08:00','2022-04-12 15:08:07');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (221903724,1,025987240,'CPB',-439,136.7215185001,'2022-11-12 12:49:52','2022-11-12 12:49:55');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (269859416,1,847054847,'MDSO',-141,223.9287630664,'2022-06-23 13:00:30','2022-06-23 13:00:36');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (168721234,1,594965059,'NYMTP',-478,457.5951970397,'2022-11-19 06:12:13','2022-11-19 06:12:14');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (771143367,1,851919263,'GBL',-369,62.1801386406,'2022-05-21 11:47:21','2022-05-21 11:47:24');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (547202198,1,560634525,'CHS',-444,345.3008315941,'2022-10-17 13:19:23','2022-10-17 13:19:28');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (515405427,1,363242941,'GOGL',-224,217.7989856559,'2022-02-24 15:23:08','2022-02-24 15:23:18');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (725147699,1,797589909,'JBSS',-247,285.022113012,'2022-03-03 14:49:25','2022-03-03 14:49:28');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (933864542,1,008819798,'TGI',-285,600.040799279,'2022-09-18 12:18:30','2022-09-18 12:18:39');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (880894172,1,240899073,'FLC',-433,203.0683595854,'2022-08-25 09:37:12','2022-08-25 09:37:15');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (085525977,1,582522518,'ASV',-173,266.3269703601,'2022-01-22 22:24:04','2022-01-22 22:24:10');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (132945899,1,125875572,'SBGI',-406,4.9860401482,'2022-10-28 03:44:38','2022-10-28 03:44:41');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (373126465,1,542027005,'NVMI',-203,470.6997699536,'2022-09-20 03:48:07','2022-09-20 03:48:16');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (585253822,1,244628470,'KMT',-127,374.4028955047,'2022-04-01 09:19:35','2022-04-01 09:19:36');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (577728734,1,550575750,'STWD',-209,372.9321654916,'2022-03-17 05:54:56','2022-03-17 05:55:06');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (237369175,1,587524774,'AIV^A',-171,291.4579045945,'2022-11-28 12:13:39','2022-11-28 12:13:40');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (819902161,1,296328766,'ABB',-424,99.8814484704,'2022-07-07 08:00:12','2022-07-07 08:00:21');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (216392122,1,712826496,'LAMR',-471,277.8374333331,'2022-08-16 03:55:02','2022-08-16 03:55:11');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (944601939,1,496707528,'ENDP',-204,491.8695386775,'2022-02-25 08:25:28','2022-02-25 08:25:32');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (842069922,1,807071871,'ESGG',-370,467.4745832951,'2022-11-04 18:05:32','2022-11-04 18:05:37');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (743946744,1,387343439,'AXTA',-256,444.067462544,'2022-02-26 05:53:14','2022-02-26 05:53:21');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (038066907,1,366846045,'EFAS',-342,240.5856947756,'2022-11-05 08:08:54','2022-11-05 08:08:57');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (319403978,1,088451452,'DRWI',-142,58.8738792894,'2022-07-10 07:26:17','2022-07-10 07:26:24');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (854604907,1,856836482,'DBD',-372,335.8564741887,'2022-10-19 16:08:09','2022-10-19 16:08:12');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (224905557,1,452572457,'CVO',-490,390.5840950084,'2022-08-15 19:04:14','2022-08-15 19:04:21');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (369880468,1,286819083,'AMCN',-246,146.9080828189,'2022-01-29 19:29:37','2022-01-29 19:29:46');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (713410483,1,948705948,'VRTV',-417,326.3657458368,'2022-05-21 05:36:28','2022-05-21 05:36:35');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (543411420,1,842398595,'CLR',-456,362.0010635524,'2022-10-26 02:02:49','2022-10-26 02:02:59');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (253552961,1,581392627,'MGYR',-334,519.9970649065,'2022-02-27 22:38:47','2022-02-27 22:38:49');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (269647479,1,026853316,'VSH',-489,361.0108930203,'2022-02-28 11:24:49','2022-02-28 11:24:50');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (852406012,1,249439234,'MFC',-487,731.6713797809,'2022-10-26 06:50:00','2022-10-26 06:50:01');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (198009183,1,136795122,'LPSN',-429,235.785458264,'2022-06-12 00:38:29','2022-06-12 00:38:39');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (184578065,1,320555851,'RTIX',-460,242.7140668878,'2022-01-19 05:37:56','2022-01-19 05:37:58');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (291843161,1,704570833,'JMPB',-367,432.08760905,'2022-10-17 12:48:36','2022-10-17 12:48:37');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (709329032,1,196894924,'FOMX',-160,87.9300451764,'2022-11-04 07:17:24','2022-11-04 07:17:25');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (717385146,1,283155297,'TBNK',-116,601.1447502228,'2022-10-14 20:05:58','2022-10-14 20:06:05');
INSERT INTO fill(fill_id,leg_no,trade_id,contract,qty,avg,placement_time,filled_time) VALUES (949796957,1,915453961,'TWNKW',-428,450.5531988293,'2022-05-16 04:56:58','2022-05-16 04:57:08')
"""

text = text.split(';')
for entry in text:
    cursor.execute(entry)

In [4]:
cursor.execute('SELECT * from strategy;')
col_headers = [x[0] for x in cursor.description]
json_data = []
the_data = cursor.fetchall()
for row in the_data:
    json_data.append(dict(zip(col_headers, row)))
json_data

[{'strategy_name': 'BullBreakout',
  'strategy_id': 1,
  'documentation_link': 'github.com/NTS-Lightning/BullBreakout',
  'launch_date': datetime.datetime(2022, 9, 6, 13, 28, 51),
  'termination_date': None},
 {'strategy_name': 'DynamicBreakoutII',
  'strategy_id': 2,
  'documentation_link': 'github.com/NTS-Lightning/DynamicBreakout',
  'launch_date': datetime.datetime(2022, 7, 30, 1, 32, 10),
  'termination_date': None},
 {'strategy_name': 'NineEMAVWAPCrossover',
  'strategy_id': 3,
  'documentation_link': 'github.com/NTS-Lightning/NineEMAVWAPCrossover',
  'launch_date': datetime.datetime(2022, 10, 11, 22, 45, 34),
  'termination_date': None},
 {'strategy_name': 'ChinaADRArbitrage',
  'strategy_id': 4,
  'documentation_link': 'github.com/NTS-Lightning/ChinaADRArbitrage',
  'launch_date': datetime.datetime(2022, 10, 2, 5, 0, 36),
  'termination_date': None},
 {'strategy_name': 'LongFTX',
  'strategy_id': 5,
  'documentation_link': 'github.com/NTS-Lightning/LongFTX',
  'launch_date': da

In [22]:
connection.commit()